# Adding Staff to Database
Camp heath records include infirmary visits for both campers and staff. The camp database only has information on campers, so a staff table needs to be cleaned and uploaded to the database. 
1. Import from file
2. Clean
3. Transform to correct types
4. Connect to database, add new table, upload data.

Because the table includes names and birthdates of staff members, this public version of the script hides any print statements that were used to look at the data. 

In [1]:
import pandas as pd

In [2]:
staff_raw = pd.read_csv('C:\\Users\\avery\\OneDrive\\health_database_docs\\staff_raw.csv')

Convert table names and data types

In [3]:
col_map = {'PersonID': 'staff_id', 
           'Full Name': 'name', 
           'Gender': 'gender', 
           'Birth Date':'birthdate', 
           'Primary Childhood ID': 'household_id', 
           'Primary Mailing Zip': 'staff_zip', 
           'Login/Email': 'email'}

staff_rename = staff_raw.rename(col_map, axis=1)

# Make birthdate into datetime
staff_rename['birthdate'] = pd.to_datetime(staff_rename['birthdate'])

print(staff_rename.dtypes)

staff_id                 int64
name                    object
gender                  object
birthdate       datetime64[ns]
household_id           float64
staff_zip               object
email                   object
dtype: object


It seems there are some records that were used to test the data system. Find and remove all records that have the word 'test' in them.

In [4]:
test_for_test = staff_rename['name'].str.contains('Test', regex=True)
staff_no_test = staff_rename[~test_for_test]

The household_id column shows that a staff member was once a camper. In the camper database, this number connects to information about their household. To serve as a foreign key to the households table in the database, this number needs to be an integer rather than a float and not contain null values. 

To make this change, first replace nulls with 0 and then convert to integer.

In [8]:
staff_no_test.loc[:, 'household_id'].fillna(0, inplace=True, axis=0)
staff_no_test.loc[:, 'household_id'] = staff_no_test.loc[:, 'household_id'].astype('int')

Finally, set the staff_id as the index.

In [6]:
staff_clean = staff_no_test.set_index('staff_id')

Now we're ready to add this to a new table in the database. For efficiency's sake, we will create the table in a IDE (DataGrip). Here is the SQL command to generate the new table:

CREATE TABLE staff( <br>
    staff_id INT PRIMARY KEY,<br>
    name VARCHAR(50), <br>
    gender VARCHAR(7), <br>
    birthdate DATE, <br>
    household_id INTEGER, <br>
    staff_zip VARCHAR(10), <br>
    email VARCHAR(65)<br>
)

Export the df as a csv to add to table.

In [7]:
staff_clean.to_csv('C:\\Users\\avery\\OneDrive\\health_database_docs\\staff_clean.csv')